<a href="https://colab.research.google.com/github/lyla-lee/TIL/blob/master/Minutes_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import io
import os
import re
import pandas as pd

# 1. PDF, HWP 파일 저장하기

In [ ]:
title_list = []
li_list = []
for i in range(1,5): #28
  url = 'https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761'
  query ={'searchCnd': '1',
          'sdate': '2005-05-01',
          'edate': '2017-12-31',
          'pageIndex': i}
  resp = requests.get(url,query)
  soup = BeautifulSoup(resp.content,'lxml')
  li = soup.find('div', class_='bdLine type2').find('ul').find('li')
  li_li = li.find_next_siblings('li')
  li_list.append(li)
  li_list.extend(li_li)  # len(li_list) =20 brings 10 pages for each page when the loop is 2, 
  
  for j in range(0,len(li_list)):
        link = li_list[j].find('div', class_='fileGoupBox').find('ul').find('li').find('a').attrs['href']
        file_title = li_list[j].find('div', class_='fileGoupBox').find('ul').find('li').find('a').text
        file_extension = re.findall('([a-z]+)', file_title)
        title = li_list[j].find('div',class_='row').find('span').find('a').find('span').find('span').text
        
        link_url = 'https://www.bok.or.kr' + link
        pdf_file = requests.get(link_url)
        if file_extension == ['pdf']:
            title_list.append(title)  # 중복 있음
            with open('{}.pdf'.format(title),'wb') as f:
                f.write(pdf_file.content)
        else:
            with open('{}.hwp'.format(title), 'wb') as f:
                f.write(pdf_file.content)
            
    
#     except:
#       link2 = li_list[j].find('div').find('div').find('a').attrs['href']
#       title = li_list[j].find('div',class_='row').find('span').find('a').find('span').find('span').text
#       link_url2 = 'https://www.bok.or.kr' + link2
#       pdf_file2 = requests.get(link_url2)
#       with open('{}.hwp'.format(title), 'wb') as f:
#         f.write(pdf_file2.content)



# 2. 파일 Text 추출

In [ ]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()
    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)
        text = retstr.getvalue()
    fp.close()
    device.close()
    retstr.close()
    return text

In [ ]:
# dictionary는 key값이 같으면 하나로 카운트하고 맨 마지막 value값을 vlue값으로 내놓는다
text_extraction={}
for i in title_list:
    text_extraction[i]=convert_pdf_to_txt('{}.pdf'.format(i))
 

# 3. Sections 뽑아내기

In [ ]:
# 섹션 분리하기
def tidy_sentences(section):
    sentence_enders = re.compile(r'((?<=[함음됨임봄짐움])(\s*\n|\.|;)|(?<=다)\.)\s*')
    splits = list((m.start(), m.end()) for m in re.finditer(sentence_enders, section))
    starts = [0] + [i[1] for i in splits]
    ends = [i[0] for i in splits]
    sentences = [section[start:end] for start, end in zip(starts[:-1], ends)]
    for i, s in enumerate(sentences):
        sentences[i] = (s.replace('\n', ' ').replace(' ', ' ')) + '.'

    text = '\n'.join(sentences) if len(sentences) > 0 else ''
    return sentences, text

In [ ]:
# 섹션 분리하기
import re 

def preprocess_minutes(minutes):

    pos = re.search('(.?국내외\s?경제\s?동향.?과 관련하여,?|\(가\).+경제전망.*|\(가\) 국내외 경제동향 및 평가)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s1 = pos.start() if pos else -1
    pos = re.search('(.?외환.?국제금융\s?동향.?과 관련하여.*|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*(일부 위원은|대부분의 위원들은)', minutes,re.MULTILINE)
    s2 = pos.start() if pos else -1
    pos = re.search('(.?금융시장\s?동향.?과 관련하여,?|\(다\) 금융시장\s?동향)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s3 = pos.start() if pos else -1
    pos = re.search('((\((다|라)\) )?.?통화정책\s?방향.?에 관한 토론,?|이상과 같은 의견\s?교환을 바탕으로.*통화정책\s?방향.*에.*토론.*)\n?', minutes,re.MULTILINE)
    s4 = pos.start() if pos else -1
    pos = re.search('(\(4\) 정부측 열석자 발언.*)\n?', minutes, re.MULTILINE)
    s5 = pos.start() if pos else -1
    pos = re.search('(\(.*\) 한국은행 기준금리 결정에 관한 위원별 의견\s?개진|이상과 같은 토론에 이어 .* 관한 위원별 의견개진이 있었음.*)\n?', minutes,re.MULTILINE)
    s6 = pos.start() if pos else -1
    positer = re.finditer('(\(\s?.*\s?\) ()(심의결과|토의결론))\n?', minutes, re.MULTILINE)
    s7 = [pos.start() for pos in positer if pos.start() > s6]
    s7 = s7[0] if s7 else -1

    # 국내외 경제동향
    bos = s1
    eos = s2
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section1, section1_txt = tidy_sentences(section)

    # 외환․국제금융 동향
    bos = s2
    eos = s3 if s3 >= 0 else s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section2, section2_txt = tidy_sentences(section)
    #print(section)

    # 금융시장 동향
    bos = s3
    eos = s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section3, section3_txt = tidy_sentences(section)

    # 통화정책방향
    bos = s4
    eos = s5 if s5 >= 0 else s6 if s6 >= 0 else s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section4, section4_txt = tidy_sentences(section)

    # 위원별 의견 개진
    bos = s6
    eos = s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section5, section5_txt = tidy_sentences(section)

    # 정부측 열석자 발언
    bos = s5
    eos = s6
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('정부측 열석자 발언', section, re.MULTILINE)
    bos = pos.end() + 1 if pos else -1
    section = section[bos:] if bos >= 0 else section
    section6, section6_txt = tidy_sentences(section)

    sections = ['Economic Situation', 'Foreign Currency', 'Financial Markets',
                'Monetary Policy', 'Participants’ Views', 'Government’s View']
    section_texts = (section1, section2, section3, section4, section5, section6)


    return sections, section_texts

In [ ]:
# 섹션 추출
result={}
for key,value in text_extraction.items():
    text_extraction[key] = preprocess_minutes(value)
    # 외환 금융 및 금융시장 동향 내용 있는 것만 분석대상에 포함 및 추출한다
    if text_extraction[key]!=(['Economic Situation', 'Foreign Currency', 'Financial Markets', 'Monetary Policy', 'Participants’ Views', 'Government’s View'], ([], [], [], [], [], [])):
        result[key] = preprocess_minutes(value)
result

{'금융통화위원회 의사록(2017년도 제22차)(2017.11.30.)': (['Economic Situation',
   'Foreign Currency',
   'Financial Markets',
   'Monetary Policy',
   'Participants’ Views',
   'Government’s View'],
  (['일부 위원은 9∼10월 실물경제 지표를 묶어서 보면 서비스업 생산이 늘어나  는 가운데 소매판매도 비교적 양호한 모습을 나타낸 것으로 보인다고 언급하고,  수출 호조에 따른 영향이 점차 소비로 파급되고 있다고 해석할 수 있는지 관련부  서의 견해를 물었으며,  이에 대해 관련부서에서는 9∼10월 소매판매의 증가는 수출 호조 이외에 연  휴 효과와 정부 정책에도 일정부분 기인한 측면이 있는 데다 최근 들어 고용시장  의 여건도 개선세가 다소 주춤한 모습을 보이고 있어 수출 증가에 따른 영향이  본격적으로 소비로 파급되고 있다고 보기는 어려울 것으로 판단된다고 답변하였  음.',
    '다만 수출의 높은 증가세가 기업의 수익성 개선, 경제지표의 호전 등을 통해  소비심리에도 긍정적으로 작용하고 있으므로 적어도 간접적으로는 소비에 영향을  미치고 있다고 해석할 수 있다고 덧붙였음.',
    '이어서 동 위원은 중국과의 사드(THAAD) 관련 해빙 분위기 등을 반영하여 내  년 가계소득이나 고용시장의 여건을 다소 낙관적으로 전망하고 있는 것은 아닌지  관련부서에 물었으며,  이에 대해 관련부서에서는 최근 한·중 관계의 변화를 고려하면 중국인 관광객  등의 회복속도가 지난 10월 전망시 예상보다 좀 더 빨라질 가능성이 있다고 답변  하였음.',
    '변하였음.',
    '또한 동 위원은 최근 관련지표의 흐름에 비추어 앞으로 반도체를 제외한 산  업에서도 설비투자가 점점 늘어날 것으로 보이는지 관련부서의 견해를 물었으며,  이에 대해 관련부서에서는 조선 등 일부 업종은 여전히 부진하겠지만 자동차,  화학 등은 신기술 분야를 중

In [ ]:
# Dataframe으로 바꿔주기 위해 list로 변환
name = []
economic_situation = []
foreign_currency = []
financial_market = []
monetary_policy = []
participants_views =[]
government_views = []

for key,value in result.items():
    name.append(key)
    economic_situation.append(result[key][1][0])
    foreign_currency.append(result[key][1][1])
    financial_market.append(result[key][1][2])
    monetary_policy.append(result[key][1][3])
    participants_views.append(result[key][1][4])
    government_views.append(result[key][1][5])
len(foreign_currency)

11

In [ ]:
minutes = pd.DataFrame(list(zip(name, economic_situation,foreign_currency,financial_market,monetary_policy,participants_views,government_views)),
                      columns=['Name','Economic_situation','Foreign_currency','Financial_market','Monetary_policy','Participants_views','Government_views'])
minutes # section 3,financial market이 비어있다면, section2에 합쳐진 것

,Name,Economic_situation,Foreign_currency,Financial_market,Monetary_policy,Participants_views,Government_views
0,금융통화위원회 의사록(2017년도 제22차)(2017.11.30.),[일부 위원은 9∼10월 실물경제 지표를 묶어서 보면 서비스업 생산이 늘어나 는 ...,[일부 위원은 최근 미국의 금융·경제상황을 보면 금년 12월 미 연준이 금리를 인...,[],[일부 위원은 수출 증가세가 높더라도 가계부채 등 구조적 요인으로 인해 민 간소비...,[일부 위원은 지난 금통위에서 언급하였던 금리인상의 요인들이 경제흐름에 의해 확인...,[]
1,금융통화위원회 의사록(2017년도 제16차)(2017.8.31.),"[일부 위원은 관련부서에서는 세계경기의 회복세 강화, 추경 집행 등 상방 리 스크...",[일부 위원은 미국의 금리 정상화 행보에도 불구하고 미 달러화가 약세기조를 지속하...,[],[일부 위원은 비록 경기가 회복되고는 있지만 GDP갭과 물가갭은 여전히 마이 너스...,[일부 위원은 이번에는 기준금리를 현 수준에서 유지하면서 경기 및 물가 관 련 불...,[]
2,금융통화위원회 의사록(2017년도 제13차)(2017.7.13.),"[일부 위원은 이번 성장전망과 관련하여 금년 전망치 달성은 무난해 보이지만, 내년...","[일부 위원은 주요국에서는 성장과 고용이 회복국면의 정점을 향해 가고 있음., 에도...",[],[일부 위원은 향후 미 연준의 보유자산 축소가 정책금리 인상과 병행하여 시 행될 ...,[일부 위원은 이번 회의에서는 기준금리를 지금의 1.25%로 동결하는 것이 바 람...,[]
3,금융통화위원회 의사록(2017년도 제10차)(2017.5.25.),[일부 위원은 금년 1/4분기 수출이 높은 증가율을 기록하였는데 단가 상승뿐만 아...,[일부 위원은 최근 국제금융시장이 안정된 모습을 지속하는 가운데 국내금융 시장도 ...,[],"[일부 위원은 낮은 수준의 중립금리 지속은 금융안정 리스크 확대, 자원배분 왜곡 ...",[일부 위원은 다음 통화정책방향 결정시까지 기준금리를 현 수준 1.25%로 유 지...,[]
4,금융통화위원회 의사록(2017년도 제1차)(2017.1.13.),[일부 위원은 대내외 여건이 앞으로의 성장경로를 논하기가 어려울 만큼 매우 불확실...,[일부 위원은 미국의 시간당평균임금 상승률이 확대되고 있어 물가상승 압력 이 높아...,"[일부 위원은 관련부서에서는 금년중 가계대출이 은행, 비은행 모두 증가세가 둔화되...",[일부 위원은 부동산경기나 금융사이클이 실물경기와 괴리되어 있는 것은 아 닌지 심...,[일부 위원은 다음 통화정책방향 결정 시까지 기준금리를 현 1.25%에서 유지 하...,[]
5,금융통화위원회 의사록(2016년도 제24차)(2016.12.15.),[일부 위원은 내년에는 재정의 성장기여도가 하락할 것으로 예상되는 가운데 정치적 ...,"[일부 위원은 미국의 재정지출 확대, 보호무 역주의 강화 등으로 미 연준의 금리인...",[일부 위원은 과거 글로벌 유동성이 은행부문을 매개로 신흥국의 자본유출입 문제를 ...,[일부 위원은 최근의 장기시장금리 상승이 주로 국내경기보다는 대외요인에 의해 주도...,"[일부 위원은 다음 정책결정시까지 현재의 기준금리 수준을 유지하되, 최근 급 증한...",[]
6,금융통화위원회 의사록(2016년도 제22차)(2016.11.11.),[일부 위원은 미 대선 결과가 우리경제에 미치 는 영향을 분석한 결과에 대해 짧은...,[일부 위원은 미국의 대선 결과가 중국경제 와 국제유가에 미치는 영향에 대해 물었...,[일부 위원은 가계대출이 예년 수준을 상회하는 높은 증가세를 지속하고 있는 가운데...,[일부 위원은 이번 주택시장관련 대책 발표 이후 주택가격이 당분간 관망세를 보이다...,[일부 위원은 이번에는 기준금리를 현 수준에서 유지하는 것이 바람직하다는 최근 세...,[]
7,금융통화위원회 의사록(2016년도 제20차)(2016.10.13.),[일부 위원은 건설투자가 내년에도 계속 늘어 날 것으로 전망되었는데 올해 건설투자...,[일부 위원은 최근 주요국의 금융시장이 대 체로 안정적인 모습을 보이고 있지만 영...,[일부 위원은 우리나라 금융시스템에서 부동산과 - 11 - 관련한 익스포저(부...,"[일부 위원은 먼저, 소비자물가 상승률이 내년 상반기 중 물가목표 수준으로 높아질...",[일부 위원은 이번 달에는 기준금리를 현 수준 1.25%에서 유지하면서 거시경 제...,[]
8,금융통화위원회 의사록(2016년도 제18차)(2016.9.9.),[일부 위원은 최근 수출이 조업일수 증가 등으 로 전년동월대비 증가하였지만 일평균...,[일부 위원은 우리나라는 달러화 유동성 사 정이 양호한 것으로 평가되지만 중국의 ...,[일부 위원은 8월 가계대출 증가내용을 보면 집 단대출의 증가폭이 축소된 반면 일...,"[일부 위원은 최근의 장단기금리차 축소에는 국내 요인보다는 글로벌 요인이, 미래 ...","[일부 위원은 기준금리를 현 수준에서 유지하는 가운데, 앞으로의 경기상황, 가계부...",[]
9,금융통화위원회 의사록(2016년도 제16차)(2016.8.11.),[일부 위원은 6월 소매판매와 설비투자가 자동 차 판매 호조 등에 힘입어 늘어나고...,[일부 위원은 7월 미국 고용지표 호조 등으 로 미국의 금리인상 재개 가능성에 대...,[일부 위원은 최근 미국 10년물 국채금리가 우리 나라 국채금리를 상회하는 역전현...,[일부 위원은 최근 몇 년 전부터 우리나라 가계소비성향이 하락하고 있으며 이는 인...,[일부 위원은 이번 달 당행의 기준금리를 다음 통화정책방향 결정시까지 현 수준으로...,[]


## Trials

In [ ]:
title_list = []
li_list = []
x=[]
for i in range(1,3):
  url = 'https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761'
  query ={'searchCnd': '1',
          'sdate': '2005-05-01',
          'edate': '2017-12-31',
          'pageIndex': i}
  resp = requests.get(url,query)
  soup = BeautifulSoup(resp.content,'lxml')
  li = soup.find('div', class_='bdLine type2').find('ul').find('li')
  li_li = li.find_next_siblings('li')
  li_list.append(li)
  li_list.extend(li_li)  # len(li_list) =20 brings 10 pages for each page when the loop is 2, 
  
  for j in range(0,len(li_list)):
        link = li_list[j].find('div', class_='fileGoupBox').find('ul').find('li').find('a').text
        file_name = re.findall('([a-z]+)', link)
        print(file_name)


['pdf']
['pdf']
['pdf']
['hwp']
['pdf']
['pdf']
['pdf']
['pdf']
['pdf']
['pdf']
['pdf']
['pdf']
['pdf']
['hwp']
['pdf']
['pdf']
['pdf']
['pdf']
['pdf']
['pdf']
['pdf']
['hwp']
['hwp']
['hwp']
['pdf']
['pdf']
['pdf']
['pdf']
['pdf']
['pdf']


In [ ]:
ddd=pd.DataFrame(result.items(), columns=['Title','Content'])

In [ ]:
g=ddd[['Content']].iloc[0].values

In [ ]:
list(g)[0].replace('\t', '').replace('\n', '').replace('\x0c', '')

AttributeError: 'tuple' object has no attribute 'replace'

In [ ]:
gg=re.findall('[가-힣]+',g[0])

In [ ]:
gg[:5]

In [ ]:
rl=''
for gf in gg:
    rl+=gf+' '

In [ ]:
rl